<a href="https://colab.research.google.com/github/Vashistsanskar/Myfirstproject/blob/main/LogisticRegression_model_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('/content/train.csv')

##Columns deleted -->Name,SibSp,Ticket,PassengerId,Cabin

##Input_features-Fare,pclass,sex,age,embarked,parch

##Target_feature-Survived


#**Small Analysis:-**
## Survival rate of adult female > adult male

---
###***Feature Selection / Extraction:***
---
#### --> New column adult_female as there alive/death ratio > alive/death ratio of adult_male Age limit [18,39] and overall also

#### --> Missing values:
 - Embarked:- by using simpleImputer(mode/most_frequent)
 - Age:- used groupby on pclass and sex then, calcuated their median values and filled accoringly.

####--> Then i applied custom binning on age column by this crieteria
    bins = [0, 15, 28, 39, 60, 80]  # Bin edges
    labels = [4.0, 1.0, 2.0, 3.0, 0.0]  # Ordinal labels as specified

By this analysis
- Age [0,17] - 100 alive-55 and dead-45 `4`

- Age [17,27] - 373 alive-103 and dead-270 `1`

- Age [27,39] - 224 alive-96 and dead-124 `2`

- Age [39,60] - 158 alive-59 and dead-99`3`

- Age [60,80] - 21 alive-4 and dead-17 `0`

####--> Custom Label encoder for Pclass,sex and Embarked `because sklearn labelencoder encoding them in different order`
- For Pclass
  - alive-217 and deas 427 --S --33% {`0-label`}

  - alive-93 and deas 75 --C -- 55% {`2-label`}

  - alive-30 and deas 47 --Q --38% {`1-label`}

####--> Polynomial Features (degree--> by gridsearccv)

####--> Logistic Regression
  - Hyperparameters are tuned by GridSearchcv.

In [ ]:
class Adultfemale_agebins(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.some=None

  def fit(self,X_train=None,y_train=None):
    return self

  def transform(self,X_train,y_train=None):
    #female_adult_col_feature
    X_train = pd.DataFrame(X_train,columns=['Fare','Pclass','Sex','Age','Embarked','Parch'])
    X_train = X_train.copy()
    mask = (X_train['Sex']=='female') & (X_train['Age']>18) & (X_train['Age']<39)
    X_train['Adult_female'] = np.where(mask,1,0)

    #age_bins_feature
    bins = [0, 15, 28, 39, 60, 80]  # Bin edges
    labels = [4.0, 1.0, 2.0, 3.0, 0.0]  # Ordinal labels as specified

    X_train['AgeGroup'] = pd.cut(X_train['Age'], bins=bins, labels=labels,include_lowest=True)
    X_train.drop(columns='Age',inplace=True)

    return X_train

In [ ]:
class Ageimpute(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.mean_age = None

  def fit(self,X_train,y_train=None):
    self.mean_age = X_train.groupby(['Pclass','Sex'])['Age'].mean()
    return self

  def transform(self,X_train,y_train=None):
        X_train = X_train.copy()
        #age_impute
        for (pclass,sex),mean_age in self.mean_age.items():
            mask = (X_train['Age'].isnull()) & (X_train['Pclass'] == pclass) & (X_train['Sex']==sex)
            X_train.loc[mask, 'Age'] = mean_age
        return X_train

In [ ]:
class pclass_sex__embarked_encode(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.one=None

  def fit(self,X_train=None,y_train=None):
    return self

  def transform(self,X_train,y_train=None):
    #pclass encode
    X_train = pd.DataFrame(X_train,columns=['Embarked','Fare','Pclass','Sex','parch','Adult_female','Age_group'])
    X_train = X_train.copy()
    X_train['Pclass'].replace({
    1:3.0,
    3:1.0
    },inplace=True)
    #sex encode
    X_train['Sex'].replace({
      'female':1.0,
      'male':0.0
    },inplace=True)

    #embarked_encode
    X_train['Embarked'].replace({
            'S':0.0,
            'Q':1.0,
            'C':2.0
        },inplace=True)

    return X_train

In [ ]:
class dropT(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.drop=None

  def fit(self,X_train=None,y_train=None):
    return self

  def transform(self,X_train,y_train=None):
    X_train = X_train.copy()

    X_train.drop(columns=['PassengerId','Ticket','Cabin','Name','SibSp'],inplace=True)

    return X_train

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns='Survived'),df['Survived'],test_size=0.2,random_state=4)

In [ ]:
trf1 = ColumnTransformer([
    ('fare_impute',SimpleImputer(),['Fare']),
    ('drops',dropT(),['PassengerId','Ticket','Name','SibSp','Cabin']),
    ('Agefillinge',Ageimpute(),['Pclass','Sex','Age','Embarked'])
],remainder='passthrough')

In [ ]:
trf2 = ColumnTransformer([
    ('adult_age_into_bins',Adultfemale_agebins(),[0,1,2,3,4,5]),
],remainder='passthrough')

In [ ]:
trf3 = ColumnTransformer([
    ('embarkImpute',SimpleImputer(strategy='most_frequent'),[3])
],remainder='passthrough')

In [ ]:
trf4 = ColumnTransformer([
    ('encoding',pclass_sex__embarked_encode(),[0,1,2,3,4,5,6])
])

In [ ]:
trf5 = LogisticRegression(C=0.5,penalty='l2',tol=0.001,max_iter=80,solver='liblinear')

In [ ]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('poly',PolynomialFeatures(3)),
    ('trf5',trf5)
])

In [ ]:
pipe.fit(X_train,y_train)

/tmp/ipython-input-24-1480020728.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Pclass'].replace({
/tmp/ipython-input-24-1480020728.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train['Pclass'].replace({
/tmp/ipython-input-24-1480020728.py:17: FutureWarning: A value is trying

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('fare_impute',
                                                  SimpleImputer(), ['Fare']),
                                                 ('drops', dropT(),
                                                  ['PassengerId', 'Ticket',
                                                   'Name', 'SibSp', 'Cabin']),
                                                 ('Agefillinge', Ageimpute(),
                                                  ['Pclass', 'Sex', 'Age',
                                                   'Embarked'])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('adult_age_into_bins',
                                                  Adultfemale...
                                                  [0, 1, 2, 3, 4, 5])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('embarkImpute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [3])])),
                ('trf4',
                 ColumnTransformer(transformers=[('encoding',
                                                  pclass_sex__embarked_encode(),
                                                  [0, 1, 2, 3, 4, 5, 6])])),
                ('poly', PolynomialFeatures(degree=3)),
                ('trf5',
                 LogisticRegression(C=0.5, max_iter=80, solver='liblinear',
                                    tol=0.001))])

In [ ]:
y_pred = pipe.predict(X_test)

/tmp/ipython-input-24-1480020728.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Pclass'].replace({
/tmp/ipython-input-24-1480020728.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train['Pclass'].replace({
/tmp/ipython-input-24-1480020728.py:17: FutureWarning: A value is trying

In [ ]:
accuracy_score(y_test,y_pred)

0.8100558659217877

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86       119
           1       0.75      0.65      0.70        60

    accuracy                           0.81       179
   macro avg       0.79      0.77      0.78       179
weighted avg       0.81      0.81      0.81       179



In [ ]:
param_grid = [
    # Grid for solver='liblinear' (supports l1 and l2)
    {
        'trf5__solver': ['liblinear'],
        'trf5__penalty': ['l1', 'l2'],
        'trf5__C': [0.001, 0.01, 0.1, 1, 10],
        'trf5__tol': [0.0001, 0.001, 0.01],
        'trf5__max_iter': [50,60,100],
        'poly__degree':[1,2,3]
    },
    # Grid for solver='saga' (supports l1, l2, elasticnet)
    {
        'trf5__solver': ['saga'],
        'trf5__penalty': ['l1', 'l2', 'elasticnet'],
        'trf5__C': [0.001, 0.01, 0.1, 1, 10],
        'trf5__tol': [0.0001, 0.001, 0.01],
        'trf5__max_iter': [50,60,100],  # saga may need more iterations
        'trf5__l1_ratio': [0.5,0.6,0.7,0.8],
        'poly__degree':[1,2,3]
    },
    # Grid for solver='lbfgs' (supports l2 only)
    {
        'trf5__solver': ['lbfgs'],
        'trf5__penalty': ['l2'],
        'trf5__C': [0.001, 0.01, 0.1, 1, 10],
        'trf5__tol': [0.0001, 0.001, 0.01],
        'trf5__max_iter': [50,60,100,200],
        'poly__degree':[1,2,3]
    }
]

In [ ]:
grid = GridSearchCV(pipe,param_grid=param_grid,cv=5,scoring='accuracy')

In [ ]:
grid.fit(X_train,y_train)

Streaming output truncated to the last 5000 lines.


  X_train['Embarked'].replace({
/tmp/ipython-input-24-1480020728.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train['Embarked'].replace({
/tmp/ipython-input-24-1480020728.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Pcla

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('fare_impute',
                                                                         SimpleImputer(),
                                                                         ['Fare']),
                                                                        ('drops',
                                                                         dropT(),
                                                                         ['PassengerId',
                                                                          'Ticket',
                                                                          'Name',
                                                                          'SibSp',
                                                                          'Cabin']),
                                                                        ('Agefillinge',
                                                                         Ageimpute(),
                                                                         ['Pclass',
                                                                          'Sex',
                                                                          'Age',
                                                                          'Embarked'])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('adul...
                          'trf5__l1_ratio': [0.5, 0.6, 0.7, 0.8],
                          'trf5__max_iter': [50, 60, 100],
                          'trf5__penalty': ['l1', 'l2', 'elasticnet'],
                          'trf5__solver': ['saga'],
                          'trf5__tol': [0.0001, 0.001, 0.01]},
                         {'poly__degree': [1, 2, 3],
                          'trf5__C': [0.001, 0.01, 0.1, 1, 10],
                          'trf5__max_iter': [50, 60, 100, 200],
                          'trf5__penalty': ['l2'], 'trf5__solver': ['lbfgs'],
                          'trf5__tol': [0.0001, 0.001, 0.01]}],
             scoring='accuracy')

In [ ]:
grid.best_params_

{'poly__degree': 3,
 'trf5__C': 0.1,
 'trf5__max_iter': 100,
 'trf5__penalty': 'l1',
 'trf5__solver': 'liblinear',
 'trf5__tol': 0.0001}

In [ ]:
grid.best_score_

np.float64(0.8062543090712107)

In [ ]:
y_pred = grid.predict(X_test)

/tmp/ipython-input-24-1480020728.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Pclass'].replace({
/tmp/ipython-input-24-1480020728.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train['Pclass'].replace({
/tmp/ipython-input-24-1480020728.py:17: FutureWarning: A value is trying

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.94      0.89       119
           1       0.85      0.65      0.74        60

    accuracy                           0.84       179
   macro avg       0.84      0.80      0.81       179
weighted avg       0.84      0.84      0.84       179



#Exporting Pipeline

In [ ]:
import pickle
pickle.dump(grid,open('pipe_grid.pkl','wb'))